# Translate `dzn` to `smt2`

### Check Versions of Tools

In [1]:
import subprocess

output = subprocess.check_output("../optimathsat/bin/optimathsat -version", shell=True, universal_newlines=True)
output

'MathSAT5 version 1.6.4 (d39a648d6bb7) (Feb 25 2020 14:02:07, gmp 6.1.2, gcc 9.2.1, 64-bit)\n'

In [61]:
output = subprocess.check_output("../libminizinc2.1.7/bin/minizinc --version", shell=True, universal_newlines=True)
output

CalledProcessError: Command '../libminizinc2.1.7/bin/minizinc --version' returned non-zero exit status 127.

In [3]:
output = subprocess.check_output("../z3/build/z3 --version", shell=True, universal_newlines=True)
output

'Z3 version 4.8.8 - 64 bit\n'

First generate the FlatZinc files using the MiniZinc tool. Make sure that a `smt2` folder is located inside `./minizinc/share/minizinc/`. Else, to enable OptiMathSAT's support for global constraints download the [smt2.tar.gz](http://optimathsat.disi.unitn.it/data/smt2.tar.gz) package and unpack it there using

```zsh
tar xf smt2.tar.gz $MINIZINC_PATH/share/minizinc/
```

If next output shows a list of `.mzn` files, then this dependency is satified.

In [52]:
output = subprocess.check_output("ls -la ../minizinc/share/minizinc/smt2/", shell=True, universal_newlines=True)
print(output)

total 292
drwxr-xr-x  2 kw kw 4096 Jan 15  2018 .
drwxr-xr-x 15 kw kw 4096 Apr 14 16:52 ..
-rw-r--r--  1 kw kw  328 Nov 13  2017 alldifferent_except_0.mzn
-rw-r--r--  1 kw kw  382 Nov 13  2017 all_different_int.mzn
-rw-r--r--  1 kw kw  396 Nov 13  2017 all_different_set.mzn
-rw-r--r--  1 kw kw  270 Nov 13  2017 all_disjoint.mzn
-rw-r--r--  1 kw kw  150 Nov 14  2017 all_equal_int.mzn
-rw-r--r--  1 kw kw  164 Nov 13  2017 all_equal_set.mzn
-rw-r--r--  1 kw kw  351 Nov 13  2017 among.mzn
-rw-r--r--  1 kw kw  305 Nov  8  2017 arg_max_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_max_int.mzn
-rw-r--r--  1 kw kw  306 Nov  8  2017 arg_min_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_min_int.mzn
-rw-r--r--  1 kw kw  480 Nov 13  2017 at_least_int.mzn
-rw-r--r--  1 kw kw  506 Nov 14  2017 at_least_set.mzn
-rw-r--r--  1 kw kw  340 Nov 13  2017 at_most1.mzn
-rw-r--r--  1 kw kw  474 Nov 13  2017 at_most_int.mzn
-rw-r--r--  1 kw kw  502 Nov 13  2017 at_most_set.mzn
-rw-r--r--  1 kw kw 1

## Transform `dzn` to `fzn` Using Model `mzn`

Then transform the desired `.dzn` file to `.fzn` using the `Mz.mzn` MiniZinc model.

In [53]:
import os

dzn_files = []
dzn_path = 'dzn-files/'

for filename in os.listdir(dzn_path):
    if filename.endswith(".dzn"):
        dzn_files.append(filename)
len(dzn_files)

3

In [54]:
fzn_path = 'fzn-files/z3/'
minizinc_base_cmd = '../minizinc/bin/minizinc \
    -Werror \
    --compile --solver org.minizinc.mzn-fzn \
    --search-dir /home/kw/minizinc/share/minizinc/smt2/ \
    mzn-model/Mz.mzn '
translate_count = 0
for dzn in dzn_files:
    translate_count += 1
    minizinc_transform_cmd = minizinc_base_cmd + dzn_path + dzn \
        + ' --output-to-file ' + fzn_path + dzn.replace('.', '-') + '.fzn'
    print(f'''({translate_count}/{len(dzn_files)}) Translating {dzn_path + dzn} to {fzn_path + dzn.replace('.', '-')}.fzn''')
    subprocess.check_output(minizinc_transform_cmd, shell=True, 
                                     universal_newlines=True)

(1/3) Translating dzn-files/A002.dzn to fzn-files/z3/A002-dzn.fzn
(2/3) Translating dzn-files/A001.dzn to fzn-files/z3/A001-dzn.fzn
(3/3) Translating dzn-files/A003.dzn to fzn-files/z3/A003-dzn.fzn


## Translate `fzn` to `smt2`

The generated `.fzn` file can be used to generate a `.smt2` file using the `fzn2smt2.py` script from this [project](https://github.com/PatrickTrentin88/fzn2omt).

In [55]:
import os

fzn_files = []
fzn_path = 'fzn-files/z3/'

for filename in os.listdir(fzn_path):
    if filename.endswith(".fzn"):
        fzn_files.append(filename)
len(fzn_files)

3

In [60]:
smt2_path = 'smt2-files/z3/'
fzn2smt2_base_cmd = '../fzn2omt/fzn2smt2.py --solver z3 --bv-enc --bv-alldifferent --smt2 '
translate_count = 0
for fzn in fzn_files:
    translate_count += 1
    fzn2smt2_transform_cmd = fzn2smt2_base_cmd + smt2_path \
    + fzn.replace('.', '-') + '.smt2 ' + fzn_path + fzn
    print(f'''({translate_count}/{len(fzn_files)}) Translating {fzn_path + fzn} to {smt2_path + fzn.replace('.', '-')}.smt2''')
    my_env = os.environ.copy()
    my_env['PATH'] = "/home/kw/optimathsat/bin/:" + my_env['PATH']
    subprocess.check_output(fzn2smt2_transform_cmd, shell=True, env=my_env,
                                     universal_newlines=True)

(1/3) Translating fzn-files/z3/A003-dzn.fzn to smt2-files/z3/A003-dzn-fzn.smt2
(2/3) Translating fzn-files/z3/A002-dzn.fzn to smt2-files/z3/A002-dzn-fzn.smt2
(3/3) Translating fzn-files/z3/A001-dzn.fzn to smt2-files/z3/A001-dzn-fzn.smt2


### Adjust `smt2` Files According to 7.5 in Paper

- Remove `OptiMathSAT` specific settings
- Add lower and upper bounds for the decision variable `pfc`

In [59]:
import os
import re

smt2_path = 'smt2-files/z3'

for file in os.listdir(smt2_path):
    if file.endswith(".smt2"):
        with open(smt2_path+'/'+file, 'r+') as myfile:
            data = "".join(line for line in myfile)

        filename = os.path.splitext(file)[0]
        print(filename)

        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.write(data)
        newFile.close()

        openFile = open(os.path.join(smt2_path, filename +'_v3.smt2'))
        data = openFile.readlines()
        additionalLines = data[-5:]
        data = data[:-5]
        openFile.close()

        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.writelines([item for item in data])
        newFile.close()

        with open(os.path.join(smt2_path, filename +'_v3.smt2'),"r") as myfile:
            data = "".join(line for line in myfile)
        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.write(data)
        matches = re.findall(r'\(define-fun .\d\d \(\) Int (\d+)\)', data)
        try:
            nbChambers = int(matches[0])
            print(nbChambers)
            k = 1
            for i in range(1,nbChambers+1):
                lb = f'''(define-fun lbound{str(i)} () Bool (> X_INTRODUCED_{str(i-1)}_ 0))\n'''
                ub = f'''(define-fun ubound{str(i)} () Bool (<= X_INTRODUCED_{str(i-1)}_ {str(nbChambers)}))\n'''
                assertLb = f'''(assert lbound{str(i)})\n'''
                assertUb = f'''(assert ubound{str(i)})\n'''

                newFile.write(lb)
                newFile.write(ub)
                newFile.write(assertLb)
                newFile.write(assertUb)
        except:
            print(f'''Check {filename} for completeness - data missing?''')
        
        newFile.writelines([item for item in additionalLines])
        newFile.close()

A003-dzn-fzn
24
A002-dzn-fzn
24
A001-dzn-fzn
20


## Test Generated `smt2` Files Using `z3`

This shoud generate the `smt2` file without any error. If this was the case then the `z3` prover can be called on a file by running


```zsh
z3 output/A001-dzn-smt2-fzn.smt2 
```

yielding something similar to

```zsh
z3 output/A001-dzn-smt2-fzn.smt2                                                       
sat
(objectives
 (obj 41881)
)
(model 
  (define-fun X_INTRODUCED_981_ () Bool
    false)
  (define-fun X_INTRODUCED_348_ () Bool
    false)
   
   .....
```

In [ ]:
result = subprocess.check_output('../z3/build/z3 smt2-files/A012-dzn-fzn.smt2', 
                        shell=True, universal_newlines=True)
print(result)